План лекции (временный блок):
* GAN - Генеративно-состязательная сеть (англ. Generative adversarial network, сокращённо GAN)
 * Контент: изображения архитектуры сети
 * Контент: Генератор (архитектура)
 * Контент: Классификатор (архитектура)
* cGAN - Условные порождающие состязательные сети CGAN (Conditional Generative Adversarial Nets)
 * Картинка принципа работы (в сеть подаются так же изображения)
 * like above: арихитектура cGAN
* DCGAN (Deep Convolutional Generative Adversarial Nets)
 * де-факто это cGAN, только для изображений, картинки похожи
* StackGAN - порождающая состязательная сеть для генерации фото-реалистичных изображений исходя из текстового описания
* LAPGAN (Laplacian Pyramid of Adversarial Networks)
* ControlGAN (Controllable Generative Adversarial Networks)
* AC-GAN Auxiliary Classifier Generative Adversarial Networks